# Various Minimizers from SciPy.Optimize

## Broyden-Fletcher-Goldfarb-Shanno algorithm (method='BFGS')
In order to converge more quickly to the solution, this routine uses the gradient of the objective function. If the gradient is not given by the user, then it is estimated using first-differences. The Broyden-Fletcher-Goldfarb-Shanno (BFGS) method typically requires fewer function calls than the simplex algorithm even when the gradient must be estimated.

To demonstrate this algorithm, the Rosenbrock function is again used. The gradient of the Rosenbrock function is the vector:
\begin{eqnarray*} 
\frac{\partial f}{\partial x_{j}} & = & \sum_{i=1}^{N}200\left(x_{i}-x_{i-1}^{2}\right)\left(\delta_{i,j}-2x_{i-1}\delta_{i-1,j}\right)-2\left(1-x_{i-1}\right)\delta_{i-1,j}.\\
& = & 200\left(x_{j}-x_{j-1}^{2}\right)-400x_{j}\left(x_{j+1}-x_{j}^{2}\right)-2\left(1-x_{j}\right).
\end{eqnarray*}

<p>This expression is valid for the interior derivatives. Special cases
are</p>

\begin{eqnarray*} 
\frac{\partial f}{\partial x_{0}} & = & -400x_{0}\left(x_{1}-x_{0}^{2}\right)-2\left(1-x_{0}\right),\\ \frac{\partial f}{\partial x_{N-1}} & = & 200\left(x_{N-1}-x_{N-2}^{2}\right).
\end{eqnarray*}


A Python function which computes this gradient is constructed by the code-segment:

In [8]:
from scipy.optimize import minimize, rosen

In [9]:
def rosen_der(x): # First Derivative of the RosonBrock function
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

This gradient information is specified in the minimize function through the jac parameter as illustrated below.

In [30]:
x0 = np.array([0.7, 1.3, 0.8, 1.9, 1.2])
res = minimize(rosen, x0, method='BFGS', jac=rosen_der,
               options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 26
         Function evaluations: 30
         Gradient evaluations: 30


In [21]:
print(res)

      fun: 4.0130879949972905e-13
 hess_inv: array([[0.00758796, 0.01243893, 0.02344025, 0.04614953, 0.09222281],
       [0.01243893, 0.02481725, 0.04712952, 0.09298607, 0.18569385],
       [0.02344025, 0.04712952, 0.09456412, 0.18674836, 0.37282072],
       [0.04614953, 0.09298607, 0.18674836, 0.37383212, 0.74621435],
       [0.09222281, 0.18569385, 0.37282072, 0.74621435, 1.49444705]])
      jac: array([-5.68982937e-06, -2.73296557e-06, -2.54520599e-06, -7.73460770e-06,
        5.78142698e-06])
  message: 'Optimization terminated successfully.'
     nfev: 30
      nit: 25
     njev: 30
   status: 0
  success: True
        x: array([1.00000004, 1.0000001 , 1.00000021, 1.00000044, 1.00000092])


## Newton-Conjugate-Gradient algorithm (method='Newton-CG')

Newton-Conjugate Gradient algorithm is a modified Newton’s method and uses a conjugate gradient algorithm to (approximately) invert the local Hessian [NW]. Newton’s method is based on fitting the function locally to a quadratic form:
<div class="math notranslate nohighlight">
\[f\left(\mathbf{x}\right)\approx f\left(\mathbf{x}_{0}\right)+\nabla f\left(\mathbf{x}_{0}\right)\cdot\left(\mathbf{x}-\mathbf{x}_{0}\right)+\frac{1}{2}\left(\mathbf{x}-\mathbf{x}_{0}\right)^{T}\mathbf{H}\left(\mathbf{x}_{0}\right)\left(\mathbf{x}-\mathbf{x}_{0}\right).\]</div>
where  is a matrix of second-derivatives (the Hessian). If the Hessian is positive definite then the local minimum of this function can be found by setting the gradient of the quadratic form to zero, resulting in
<div class="math notranslate nohighlight">
\[\mathbf{x}_{\textrm{opt}}=\mathbf{x}_{0}-\mathbf{H}^{-1}\nabla f.\]</div>
The inverse of the Hessian is evaluated using the conjugate-gradient method. An example of employing this method to minimizing the Rosenbrock function is given below. To take full advantage of the Newton-CG method, a function which computes the Hessian must be provided. The Hessian matrix itself does not need to be constructed, only a vector which is the product of the Hessian with an arbitrary vector needs to be available to the minimization routine. As a result, the user can provide either a function to compute the Hessian matrix, or a function to compute the product of the Hessian with an arbitrary vector.

Full Hessian example:
The Hessian of the Rosenbrock function is
<div class="math notranslate nohighlight">
 \begin{eqnarray*} H_{ij}=\frac{\partial^{2}f}{\partial x_{i}\partial x_{j}} &amp; = &amp; 200\left(\delta_{i,j}-2x_{i-1}\delta_{i-1,j}\right)-400x_{i}\left(\delta_{i+1,j}-2x_{i}\delta_{i,j}\right)-400\delta_{i,j}\left(x_{i+1}-x_{i}^{2}\right)+2\delta_{i,j},\\  &amp; = &amp; \left(202+1200x_{i}^{2}-400x_{i+1}\right)\delta_{i,j}-400x_{i}\delta_{i+1,j}-400x_{i-1}\delta_{i-1,j},\end{eqnarray*}</div>

if $(i,j \in [1,N-2] )$ with $(i,j\in[0,N-1])$ defining the  matrix. Other non-zero entries of the matrix are
<div class="math notranslate nohighlight">
\begin{eqnarray*} \frac{\partial^{2}f}{\partial x_{0}^{2}} & = & 1200x_{0}^{2}-400x_{1}+2,\\ 
\frac{\partial^{2}f}{\partial x_{0}\partial x_{1}}=\frac{\partial^{2}f}{\partial x_{1}\partial x_{0}} & = & -400x_{0},\\ \frac{\partial^{2}f}{\partial x_{N-1}\partial x_{N-2}}=\frac{\partial^{2}f}{\partial x_{N-2}\partial x_{N-1}} & = & -400x_{N-2},\\ 
\frac{\partial^{2}f}{\partial x_{N-1}^{2}} & = & 200.\end{eqnarray*}</div>

For example, the Hessian when $N =5$ is
<div class="math notranslate nohighlight">
\[\begin{split}\mathbf{H}=
\begin{bmatrix} 
1200x_{0}^{2}-400x_{1}+2 & -400x_{0} & 0 & 0 & 0\\
-400x_{0} & 202+1200x_{1}^{2}-400x_{2} & -400x_{1} & 0 & 0\\ 
0 & -400x_{1} & 202+1200x_{2}^{2}-400x_{3} & -400x_{2} & 0\\
0 &  & -400x_{2} & 202+1200x_{3}^{2}-400x_{4} & -400x_{3}\\ 0 & 0 & 0 & -400x_{3} & 200
\end{bmatrix}.
\end{split}\]</div>

The code which computes this Hessian along with the code to minimize the function using Newton-CG method is shown in the following example:

In [22]:
def rosen_hess(x): # Hessian Matrix of the Rosenbrock function
    x = np.asarray(x)
    H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
    diagonal = np.zeros_like(x)
    diagonal[0] = 1200*x[0]**2-400*x[1]+2
    diagonal[-1] = 200
    diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
    H = H + np.diag(diagonal)
    return H

In [23]:
res = minimize(rosen, x0, method='Newton-CG',
               jac=rosen_der, hess=rosen_hess,
               options={'xtol': 1e-8, 'disp': True})
print(res)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 24
         Function evaluations: 33
         Gradient evaluations: 33
         Hessian evaluations: 24
     fun: 3.5306674342205174e-17
     jac: array([ 2.68742124e-08,  9.26710155e-08,  3.70109749e-07,  1.48479130e-06,
       -8.52636894e-07])
 message: 'Optimization terminated successfully.'
    nfev: 33
    nhev: 24
     nit: 24
    njev: 33
  status: 0
 success: True
       x: array([1.        , 1.        , 1.        , 0.99999999, 0.99999999])


Hessian product example:
For larger minimization problems, storing the entire Hessian matrix can consume considerable time and memory. The Newton-CG algorithm only needs the product of the Hessian times an arbitrary vector. As a result, the user can supply code to compute this product rather than the full Hessian by giving a hess function which take the minimization vector as the first argument and the arbitrary vector as the second argument (along with extra arguments passed to the function to be minimized). If possible, using Newton-CG with the Hessian product option is probably the fastest way to minimize the function.

In this case, the product of the Rosenbrock Hessian with an arbitrary vector is not difficult to compute. If $\mathbf{p}$ is the arbitrary vector, then $\mathbf{H}\left(\mathbf{x}\right)\mathbf{p}$ has elements:

<div class="math notranslate nohighlight">
\[\begin{split}\mathbf{H}\left(\mathbf{x}\right)\mathbf{p}=\begin{bmatrix} \left(1200x_{0}^{2}-400x_{1}+2\right)p_{0}-400x_{0}p_{1}\\ \vdots\\ -400x_{i-1}p_{i-1}+\left(202+1200x_{i}^{2}-400x_{i+1}\right)p_{i}-400x_{i}p_{i+1}\\ \vdots\\ -400x_{N-2}p_{N-2}+200p_{N-1}\end{bmatrix}.\end{split}\]</div>


Code which makes use of this Hessian product to minimize the Rosenbrock function using minimize follows:

In [24]:
def rosen_hess_p(x, p):
    x = np.asarray(x)
    Hp = np.zeros_like(x)
    Hp[0] = (1200*x[0]**2 - 400*x[1] + 2)*p[0] - 400*x[0]*p[1]
    Hp[1:-1] = -400*x[:-2]*p[:-2]+(202+1200*x[1:-1]**2-400*x[2:])*p[1:-1] \
               -400*x[1:-1]*p[2:]
    Hp[-1] = -400*x[-2]*p[-2] + 200*p[-1]
    return Hp

In [25]:
res = minimize(rosen, x0, method='Newton-CG',
               jac=rosen_der, hessp=rosen_hess_p,
               options={'xtol': 1e-8, 'disp': True})
print(res)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 24
         Function evaluations: 33
         Gradient evaluations: 33
         Hessian evaluations: 66
     fun: 3.5306674342205174e-17
     jac: array([ 2.68742124e-08,  9.26710155e-08,  3.70109749e-07,  1.48479130e-06,
       -8.52636894e-07])
 message: 'Optimization terminated successfully.'
    nfev: 33
    nhev: 66
     nit: 24
    njev: 33
  status: 0
 success: True
       x: array([1.        , 1.        , 1.        , 0.99999999, 0.99999999])


According to [NW] p. 170 the Newton-CG algorithm can be inefficient when the Hessian is ill-conditioned because of the poor quality search directions provided by the method in those situations. The method trust-ncg, according to the authors, deals more effectively with this problematic situation and will be described next.

## Trust-Region Newton-Conjugate-Gradient Algorithm (method='trust-ncg')
The Newton-CG method is a line search method: it finds a direction of search minimizing a quadratic approximation of the function and then uses a line search algorithm to find the (nearly) optimal step size in that direction. An alternative approach is to, first, fix the step size limit $\Delta$ and then find the
optimal step <span class="math notranslate nohighlight">$\mathbf{p}$ inside the given trust-radius by solving the following quadratic subproblem:
<div class="math notranslate nohighlight">
\begin{eqnarray*}
   \min_{\mathbf{p}} f\left(\mathbf{x}_{k}\right)+\nabla f\left(\mathbf{x}_{k}\right)\cdot\mathbf{p}+\frac{1}{2}\mathbf{p}^{T}\mathbf{H}\left(\mathbf{x}_{k}\right)\mathbf{p};&\\
   \text{subject to: } \|\mathbf{p}\|\le \Delta. &;
 \end{eqnarray*}</div>

The solution is then updated <span class="math notranslate nohighlight">$\mathbf{x}_{k+1} = \mathbf{x}_{k} + \mathbf{p}$</span> and the trust-radius $\Delta$ is adjusted according to the degree of agreement of the quadratic model with the real function. This family of methods is known as trust-region methods. The trust-ncg algorithm is a trust-region method that uses a conjugate gradient algorithm to solve the trust-region subproblem [NW].

Full Hessian example:

In [26]:
res = minimize(rosen, x0, method='trust-ncg',
               jac=rosen_der, hess=rosen_hess,
               options={'gtol': 1e-8, 'disp': True})
print(res)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 20
         Function evaluations: 21
         Gradient evaluations: 20
         Hessian evaluations: 19
     fun: 1.232595164407831e-30
    hess: array([[ 802., -400.,    0.,    0.,    0.],
       [-400., 1002., -400.,    0.,    0.],
       [   0., -400., 1002., -400.,    0.],
       [   0.,    0., -400., 1002., -400.],
       [   0.,    0.,    0., -400.,  200.]])
     jac: array([-0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  4.44089210e-14,
       -2.22044605e-14])
 message: 'Optimization terminated successfully.'
    nfev: 21
    nhev: 19
     nit: 20
    njev: 20
  status: 0
 success: True
       x: array([1., 1., 1., 1., 1.])


Hessian Product example:

In [27]:
res = minimize(rosen, x0, method='trust-ncg',
               jac=rosen_der, hessp=rosen_hess_p,
               options={'gtol': 1e-8, 'disp': True})
print(res)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 20
         Function evaluations: 21
         Gradient evaluations: 20
         Hessian evaluations: 74
     fun: 1.232595164407831e-30
     jac: array([-0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  4.44089210e-14,
       -2.22044605e-14])
 message: 'Optimization terminated successfully.'
    nfev: 21
    nhev: 74
     nit: 20
    njev: 20
  status: 0
 success: True
       x: array([1., 1., 1., 1., 1.])


**References:**

F. Lenders, C. Kirches, A. Potschka: “trlib: A vector-free implementation of the GLTR method for iterative solution of the trust region problem”, https://arxiv.org/abs/1611.04718

GLTR
N. Gould, S. Lucidi, M. Roma, P. Toint: “Solving the Trust-Region Subproblem using the Lanczos Method”, SIAM J. Optim., 9(2), 504–525, (1999). https://doi.org/10.1137/S1052623497322735

## Trust-Region Nearly Exact Algorithm (method='trust-exact')

All methods ``Newton-CG``, ``trust-ncg`` and ``trust-krylov`` are suitable for dealing with large-scale problems (problems with thousands of variables). That is because the conjugate gradient algorithm approximately solve the trust-region subproblem (or invert the Hessian) by iterations without the explicit Hessian factorization. Since only the product of the Hessian with an arbitrary vector is needed, the algorithm is specially suited for dealing with sparse Hessians, allowing low storage requirements and significant time savings for those sparse problems.

For medium-size problems, for which the storage and factorization cost of the Hessian are not critical, it is possible to obtain a solution within fewer iteration by solving the trust-region subproblems almost exactly. To achieve that, a certain nonlinear equations is solved iteratively for each quadratic subproblem [CGT]. This solution requires usually 3 or 4 Cholesky factorizations of the Hessian matrix. As the result, the method converges in fewer number of iterations and takes fewer evaluations of the objective function than the other implemented trust-region methods. The Hessian product option is not supported by this algorithm. An example using the Rosenbrock function follows:

In [29]:
res = minimize(rosen, x0, method='trust-exact',
               jac=rosen_der, hess=rosen_hess,
               options={'gtol': 1e-8, 'disp': True})
print(res)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 13
         Function evaluations: 14
         Gradient evaluations: 13
         Hessian evaluations: 14
     fun: 1.534303144849083e-22
    hess: array([[ 802., -400.,    0.,    0.,    0.],
       [-400., 1002., -400.,    0.,    0.],
       [   0., -400., 1002., -400.,    0.],
       [   0.,    0., -400., 1002., -400.],
       [   0.,    0.,    0., -400.,  200.]])
     jac: array([ 7.50333129e-12,  2.58948418e-11,  1.04757092e-10,  4.21017887e-10,
       -2.39808173e-10])
 message: 'Optimization terminated successfully.'
    nfev: 14
    nhev: 14
     nit: 13
    njev: 13
  status: 0
 success: True
       x: array([1., 1., 1., 1., 1.])


## Constrained minimization of multivariate scalar functions (minimize)

The minimize function provides algorithms for constrained minimization, namely '``trust-constr``' , '``SLSQP``' and '``COBYLA``'. They require the constraints to be defined using slightly different structures. The method 'trust-constr' requires the constraints to be defined as a sequence of objects LinearConstraint and NonlinearConstraint. Methods 'SLSQP' and 'COBYLA', on the other hand, require constraints to be defined as a sequence of dictionaries, with keys type, fun and jac.

As an example let us consider the constrained minimization of the Rosenbrock function:
<div class="math notranslate nohighlight">
\begin{eqnarray*} \min_{x_0, x_1} & ~~100\left(x_{1}-x_{0}^{2}\right)^{2}+\left(1-x_{0}\right)^{2} &\\
                  \text{subject to: } & x_0 + 2 x_1 \leq 1 & \\
                                      & x_0^2 + x_1 \leq 1  & \\
                                      & x_0^2 - x_1 \leq 1  & \\
                                      & 2 x_0 + x_1 = 1 & \\
                                      & 0 \leq  x_0  \leq 1 & \\
                                      & -0.5 \leq  x_1  \leq 2.0. &
\end{eqnarray*}</div>

This optimization problem has the unique solution $[x_0, x_1] = [0.4149,~ 0.1701]$, for which only the first and fourth constraints are active.

Trust-Region Constrained Algorithm (method='trust-constr')
The trust-region constrained method deals with constrained minimization problems of the form:

<div class="math notranslate nohighlight">
\begin{eqnarray*} \min_x & f(x) & \\
       \text{subject to: } & ~~~ c^l  \leq c(x) \leq c^u, &\\
        &  x^l  \leq x \leq x^u. & 
\end{eqnarray*}</div>

When $c^l_j = c^u_j$ the method reads the $j$-th constraint as an equality constraint and deals with it accordingly. Besides that, one-sided constraint can be specified by setting the upper or lower bound to ``np.inf`` with the appropriate sign.

The implementation is based on [EQSQP] for equality-constraint problems and on [TRIP] for problems with inequality constraints. Both are trust-region type algorithms suitable for large-scale problems.

#### Defining Bounds Constraints:¶

The bound constraints $0 \leq  x_0  \leq 1$ and $-0.5 \leq  x_1  \leq 2.0$ are defined using a Bounds object.

In [31]:
from scipy.optimize import Bounds
bounds = Bounds([0, -0.5], [1.0, 2.0])

#### Defining Linear Constraints:
The constraints $x_0+2x_1 \leq 1$ and $2x_0 + x_1 = 1$ can be written in the linear constraint standard format:

<div class="math notranslate nohighlight">
  \begin{equation*} \begin{bmatrix}-\infty \\1\end{bmatrix} \leq
   \begin{bmatrix} 1& 2 \\ 2& 1\end{bmatrix}
    \begin{bmatrix} x_0 \\x_1\end{bmatrix} \leq
     \begin{bmatrix} 1 \\ 1\end{bmatrix},\end{equation*}</div>


and defined using a [LinearConstraint](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.LinearConstraint.html#scipy.optimize.LinearConstraint) object.

In [39]:
from scipy.optimize import LinearConstraint
linear_constraint = LinearConstraint([[1, 2], [2, 1]], [-np.inf, 1], [1, 1])

#### Defining Nonlinear Constraints:

The nonlinear constraint:
<div class="math notranslate nohighlight">
  \begin{equation*} c(x) =
  \begin{bmatrix} x_0^2 + x_1 \\ x_0^2 - x_1\end{bmatrix}
   \leq
   \begin{bmatrix} 1 \\ 1\end{bmatrix}, \end{equation*}</div>
with Jacobian matrix:
<div class="math notranslate nohighlight">
\begin{equation*} J(x) =
  \begin{bmatrix} 2x_0 & 1 \\ 2x_0 & -1
   \end{bmatrix},
\end{equation*}</div>

and linear combination of the Hessians:
<div class="math notranslate nohighlight">
  \begin{equation*} H(x, v) = \sum_{i=0}^1 v_i \nabla^2 c_i(x) =
  v_0\begin{bmatrix} 2 & 0 \\ 0 & 0\end{bmatrix} +
  v_1\begin{bmatrix} 2 & 0 \\ 0 & 0\end{bmatrix},
  \end{equation*}</div>

is defined using a [NonlinearConstraint](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.NonlinearConstraint.html#scipy.optimize.NonlinearConstraint) object.

In [32]:
def cons_f(x):
    return [x[0]**2 + x[1], x[0]**2 - x[1]]
def cons_J(x):
    return [[2*x[0], 1], [2*x[0], -1]]
def cons_H(x, v):
    return v[0]*np.array([[2, 0], [0, 0]]) + v[1]*np.array([[2, 0], [0, 0]])
from scipy.optimize import NonlinearConstraint
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 1, jac=cons_J, hess=cons_H)

Alternatively, it is also possible to define the Hessian $H(x, v)$ as a sparse matrix,

In [33]:
from scipy.sparse import csc_matrix
def cons_H_sparse(x, v):
    return v[0]*csc_matrix([[2, 0], [0, 0]]) + v[1]*csc_matrix([[2, 0], [0, 0]])
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 1,
                                           jac=cons_J, hess=cons_H_sparse)

or as a [LinearOperator](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.LinearOperator.html#scipy.sparse.linalg.LinearOperator) object.

In [34]:
from scipy.sparse.linalg import LinearOperator
def cons_H_linear_operator(x, v):
    def matvec(p):
        return np.array([p[0]*2*(v[0]+v[1]), 0])
    return LinearOperator((2, 2), matvec=matvec)
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 1,
                                          jac=cons_J, hess=cons_H_linear_operator)

When the evaluation of the Hessian $H(x,v)$ is difficult to implement or computationally infeasible, one may use **HessianUpdateStrategy**. Currently available strategies are **BFGS** and **SR1**.

In [35]:
from scipy.optimize import BFGS
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 1, jac=cons_J, hess=BFGS())

Alternatively, the Hessian may be approximated using finite differences.

In [36]:
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 1, jac=cons_J, hess='2-point')

The Jacobian of the constraints can be approximated by finite differences as well. In this case, however, the Hessian cannot be computed with finite differences and needs to be provided by the user or defined using [HessianUpdateStrategy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.HessianUpdateStrategy.html#scipy.optimize.HessianUpdateStrategy).

In [37]:
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 1, jac='2-point', hess=BFGS())

### Solving the Optimization Problem:
The optimization problem is solved using:

In [40]:
x0 = np.array([0.5, 0])
res = minimize(rosen, x0, method='trust-constr', jac=rosen_der, hess=rosen_hess,
               constraints=[linear_constraint, nonlinear_constraint],
               options={'verbose': 1}, bounds=bounds)
print(res)

`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 8, CG iterations: 7, optimality: 2.99e-09, constraint violation: 0.00e+00, execution time: 0.055 s.
 barrier_parameter: 0.00016000000000000007
 barrier_tolerance: 0.00016000000000000007
          cg_niter: 7
      cg_stop_cond: 1
            constr: [array([0.75516406, 1.        ]), array([0.34228899, 0.00207024]), array([0.41494531, 0.17010937])]
       constr_nfev: [0, 24, 0]
       constr_nhev: [0, 0, 0]
       constr_njev: [0, 0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 0.05500531196594238
               fun: 0.3427175756422305
              grad: array([-0.82649483, -0.41404799])
               jac: [array([[1, 2],
       [2, 1]]), array([[ 0.82989064,  1.        ],
       [ 0.82989064, -1.        ]]), array([[1., 0.],
       [0., 1.]])]
   lagrangian_grad: array([ 1.49481413e-09, -2.98962822e-09])
           message: '`gtol` termination condition is satisfied.'

or a Hessian-vector product through the parameter ``hessp``.

In [41]:
res = minimize(rosen, x0, method='trust-constr', jac=rosen_der, hessp=rosen_hess_p,
               constraints=[linear_constraint, nonlinear_constraint],
               options={'verbose': 1}, bounds=bounds)
print(res)

`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 8, CG iterations: 7, optimality: 2.99e-09, constraint violation: 0.00e+00, execution time: 0.029 s.
 barrier_parameter: 0.00016000000000000007
 barrier_tolerance: 0.00016000000000000007
          cg_niter: 7
      cg_stop_cond: 1
            constr: [array([0.75516406, 1.        ]), array([0.34228899, 0.00207024]), array([0.41494531, 0.17010937])]
       constr_nfev: [0, 24, 0]
       constr_nhev: [0, 0, 0]
       constr_njev: [0, 0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 0.028998851776123047
               fun: 0.3427175756422305
              grad: array([-0.82649483, -0.41404799])
               jac: [array([[1, 2],
       [2, 1]]), array([[ 0.82989064,  1.        ],
       [ 0.82989064, -1.        ]]), array([[1., 0.],
       [0., 1.]])]
   lagrangian_grad: array([ 1.49481413e-09, -2.98962822e-09])
           message: '`gtol` termination condition is satisfied.

Alternatively, the first and second derivatives of the objective function can be approximated. For instance, the Hessian can be approximated with SR1 quasi-Newton approximation and the gradient with finite differences.

In [42]:
from scipy.optimize import SR1
res = minimize(rosen, x0, method='trust-constr',  jac="2-point", hess=SR1(),
               constraints=[linear_constraint, nonlinear_constraint],
               options={'verbose': 1}, bounds=bounds)
print(res)

`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 24, CG iterations: 7, optimality: 4.40e-09, constraint violation: 0.00e+00, execution time: 0.026 s.
 barrier_parameter: 0.00016000000000000007
 barrier_tolerance: 0.00016000000000000007
          cg_niter: 7
      cg_stop_cond: 1
            constr: [array([0.75516406, 1.        ]), array([0.34228898, 0.00207024]), array([0.41494531, 0.17010937])]
       constr_nfev: [0, 24, 0]
       constr_nhev: [0, 0, 0]
       constr_njev: [0, 0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 0.026001930236816406
               fun: 0.3427175756442948
              grad: array([-0.82649317, -0.41404723])
               jac: [array([[1, 2],
       [2, 1]]), array([[ 0.82989064,  1.        ],
       [ 0.82989064, -1.        ]]), array([[1., 0.],
       [0., 1.]])]
   lagrangian_grad: array([ 2.20000143e-09, -4.40000288e-09])
           message: '`gtol` termination condition is satisfied

**References:**

Byrd, Richard H., Mary E. Hribar, and Jorge Nocedal. 1999. An interior point algorithm for large-scale nonlinear programming. SIAM Journal on Optimization 9.4: 877-900.

EQSQP
Lalee, Marucha, Jorge Nocedal, and Todd Plantega. 1998. On the implementation of an algorithm for large-scale equality constrained optimization. SIAM Journal on Optimization 8.3: 682-706.

## Sequential Least SQuares Programming (SLSQP) Algorithm (method='SLSQP')

The SLSQP method deals with constrained minimization problems of the form:
<div class="math notranslate nohighlight">
\begin{eqnarray*} \min_x & f(x) \\
 \text{subject to: } & c_j(x) =  0  ,  & j \in \mathcal{E}\\
  & c_j(x) \geq 0  ,  & j \in \mathcal{I}\\
  &  \text{lb}_i  \leq x_i \leq \text{ub}_i , & i = 1,...,N. 
\end{eqnarray*}</div>
        
Where $\mathcal{E}$ or $\mathcal{I}$ are sets of indices containing equality and inequality constraints.

Both linear and nonlinear constraints are defined as dictionaries with keys ``type``, ``fun`` and ``jac``.

In [43]:
ineq_cons = {'type': 'ineq',
             'fun' : lambda x: np.array([1 - x[0] - 2*x[1],
                                         1 - x[0]**2 - x[1],
                                         1 - x[0]**2 + x[1]]),
             'jac' : lambda x: np.array([[-1.0, -2.0],
                                         [-2*x[0], -1.0],
                                         [-2*x[0], 1.0]])}
eq_cons = {'type': 'eq',
           'fun' : lambda x: np.array([2*x[0] + x[1] - 1]),
           'jac' : lambda x: np.array([2.0, 1.0])}

And the optimization problem is solved with:

In [44]:
x0 = np.array([0.5, 0])
res = minimize(rosen, x0, method='SLSQP', jac=rosen_der,
               constraints=[eq_cons, ineq_cons], options={'ftol': 1e-9, 'disp': True},
               bounds=bounds)
print(res)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.34271757499419825
            Iterations: 4
            Function evaluations: 5
            Gradient evaluations: 4
     fun: 0.34271757499419825
     jac: array([-0.82676145, -0.41372864])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([0.41494475, 0.1701105 ])


Most of the options available for the method '``trust-constr``' are not available for '``SLSQP``'.